$$\textrm{Joaquin Peñuela Parra}$$
$$\textrm{Universidad de los Andes}$$
$$\textrm{Grupo de Física de Altas Energías: Fenomenología de Partículas}$$

$\textbf{Preliminares}$ 

Las librerías que se usan en este capítulo son las siguientes: 

In [1]:
import os, sys

Path_Tutorials = os.path.dirname(os.path.realpath('Capitulo_2_Lectura_de_los_archivos_root.ipynb'))
Path_Pheno_BSM = os.path.dirname(Path_Tutorials)
sys.path.append(Path_Pheno_BSM)
sys.path.append(f'{Path_Pheno_BSM}/Leptoquarks_searches/03_delphes_preselection')

from ROOT import TChain #Para poder leer los datos del archivo .root

from delphes_reader import Quiet #Permite ignorar algunas excepciones que ocurren al leer archivos de Delphes
from delphes_reader import DelphesLoader 
from delphes_reader.clasificator import get_muons
from delphes_reader import root_analysis
import event_selection

import pandas as pd

personal_folder = f'{Path_Pheno_BSM}/Tutorials'

Welcome to JupyROOT 6.26/06


$\textbf{Definir las rutas de los archivos .root para cada proceso}$ 

Ahora debemos leer todos los archivos .root de la señal y también del background, para esto necesitamos definir esas rutas primero: 

In [2]:
#Carguemos las rutas de cada background y de la señal

Paths = {}

#Paths .root de señal:

signal = f'{personal_folder}/Data_Z/Events'
Paths['z'] = []

for i in range(1,7):
    Paths['z'].append(f'{signal}/run_0{i}/tag_1_delphes_events.root') 

#Paths .root de backgrounds:

BKGs = ["w_jets", "ww", "wz", "zz", "ttbar", "stop"]

for BKG in BKGs:
    
    Delphes_Process = DelphesLoader(BKG) #Se crea un objeto de la clase DelphesLoader correspondiente al proceso BKG
    Paths[BKG]= Delphes_Process.Forest #Forest es una función que extrae todas las rutas .root de los procesos BKG que están alojados en el servidor

w_jets imported with
599 trees!/Madgraph_Simulations/SM_Backgrounds/w_jets/

ww imported with
250 trees!/Madgraph_Simulations/SM_Backgrounds/ww/

wz imported with
200 trees!/Madgraph_Simulations/SM_Backgrounds/wz/

zz imported with
200 trees!/Madgraph_Simulations/SM_Backgrounds/zz/

ttbar imported with
500 trees!/Madgraph_Simulations/SM_Backgrounds/ttbar/

stop imported with
232 trees!/Madgraph_Simulations/SM_Backgrounds/stop



Así, Paths es un directorio donde están todas las rutas de los .root, simplemente hay que usar sus keys para tener todas las rutas que uno necesite:

In [3]:
Paths.keys()

dict_keys(['z', 'w_jets', 'ww', 'wz', 'zz', 'ttbar', 'stop'])

**Por ejemplo:**

In [4]:
Paths['z']

['/disco4/personal_folders/Joaquin/Data_Z/Events/run_01/tag_1_delphes_events.root',
 '/disco4/personal_folders/Joaquin/Data_Z/Events/run_02/tag_1_delphes_events.root',
 '/disco4/personal_folders/Joaquin/Data_Z/Events/run_03/tag_1_delphes_events.root',
 '/disco4/personal_folders/Joaquin/Data_Z/Events/run_04/tag_1_delphes_events.root',
 '/disco4/personal_folders/Joaquin/Data_Z/Events/run_05/tag_1_delphes_events.root',
 '/disco4/personal_folders/Joaquin/Data_Z/Events/run_06/tag_1_delphes_events.root']

$\textbf{Leer los archivos .root y crear archivos .csv con la información de interés}$

En general uno cada vez que fuera a hacer el análisis cinemático podría leer directamente los archivos .root; sin embargo, estos contienen demasiada información, es más eficiente, leerlos una vez implementando los cortes que uno quiera y construir un archivo .csv con los datos que son de mayor interés, esos archivos .csv son mucho más fáciles de leer.

Dicho esto, creemos las carpetas en donde guardaremos esos .csv

In [5]:
!mkdir -p {personal_folder}/CSV_Z_Analisis/Data_muon0 #Contendrá CSVs con las variables cinemáticas del muón de mayor momento
!mkdir -p {personal_folder}/CSV_Z_Analisis/Data_muon1 #Contendrá CSVs con las variables cinemáticas del segundo muón de mayor momento
!mkdir -p {personal_folder}/CSV_Z_Analisis/Data_correlation #Contendrá CSVs con las variables cinemáticas que correlacionan ambos muones
!mkdir -p {personal_folder}/CSV_Z_Analisis/Data_Z #Contendrá CSVs con las variables cinemáticas que correlacionan ambos muones

Ahora debemos crear un directorio que nos permita guardar el número de eventos que quedan a medida que se van aplicando cortes a los datos, para esto se puede utilizar lo siguiente: 

In [6]:
cut_flows = {}
cut_flows['Total'] = {}

def update_cut_flows(signal, etiqueta):
    event_selection.count_event(cut_flows['Total'], etiqueta)
    event_selection.count_event(cut_flows[signal], etiqueta) 

Notemos que aquí se está usando la función count_event definida en Delphes_Reader. Recorramos algunos eventos solo para ver cómo se puede usar todo esto: 

In [7]:
with Quiet(): #Esto evita que salgan excepciones que usualmente salen en archivos de Delphes

    for signal in Paths:
        signal_paths = Paths[signal]
        cut_flows[signal] = {} #Para cada señal llevaremos el registro de cut_flows

        for path in signal_paths:
            tree = TChain("Delphes;1") #arbol de datos, ahora debemos añadirle la ruta
            tree.Add(path) #Ahora tree es el arbol de datos de la ruta path
            
            j = 0
            for event in tree:
                j = j + 1
                if(j == 1001): break #Solo se recorren 1000 para practicar en el tutorial
                
                update_cut_flows(signal, 'Todos') 
                
                muons = get_muons(event) #Lista de los muones de un evento en el arbol
                
                #Cuts:

                if (len(muons) < 2): continue #Se descarta si hay menos de dos muones
                update_cut_flows(signal, 'Al menos 2 muones') 
                                
                if(muons[0].GetCharge()*muons[1].GetCharge() > 0): continue #Carga opuesta
                update_cut_flows(signal, 'Carga opuesta') 
                
                if (muons[0].pt() < 30): continue 
                update_cut_flows(signal, 'p_T[0] > 30 GeV') 
                
                if (muons[1].pt() < 20): continue 
                update_cut_flows(signal, 'p_T[1] > 20 GeV') 
                
            break #Asi solo se corre la primera path de cada señal

Así, cut_flows estará dado de la siguiente forma:

In [8]:
pd.DataFrame(cut_flows)

,Total,z,w_jets,ww,wz,zz,ttbar,stop
Todos,7000,1000,1000.0,1000,1000,1000,1000,1000
Al menos 2 muones,747,699,NaN,4,16,22,4,2
Carga opuesta,745,699,NaN,4,14,22,4,2
p_T[0] > 30 GeV,672,628,NaN,4,13,22,3,2
p_T[1] > 20 GeV,644,609,NaN,4,11,16,2,2


Con esto claro, ahora si extraigamos la información cinemática de los muones y guardémosla en archivos CSV.

In [9]:
cut_flows = {}
cut_flows['Total'] = {}

with Quiet(): #Esto evita que salgan excepciones que usualmente salen en archivos de Delphes

    for signal in Paths:
        signal_paths = Paths[signal]
        cut_flows[f'{signal}'] = {} #Para cada señal llevaremos el registro de cut_flows
        
        #Debemos crear DataFrames donde se pueda guardar la información cinemática de nuestro interés, en este caso corresponde a las variables cinematicas de los dos muones, las variables que los relacionan y las variables del z, la particula reconstruida con esos muones
        Data_muon0 = pd.DataFrame()
        Data_muon1 = pd.DataFrame()
        Data_correlation = pd.DataFrame()
        Data_Z = pd.DataFrame()

        for path in signal_paths:
            tree = TChain("Delphes;1") #arbol de datos, ahora debemos añadirle la ruta
            tree.Add(path) #Ahora tree es el arbol de datos de la ruta path
            
            for event in tree:
                update_cut_flows(signal, 'Todos') 
                
                muons = get_muons(event) #Lista de los muones de un evento en el arbol
                
                #Cuts:

                if (len(muons) < 2): continue #Se descarta si hay menos de dos muones
                update_cut_flows(signal, 'Al menos 2 muones') 
                
                if (len(muons) != 2): continue #Exactamente 2 muones
                update_cut_flows(signal, 'Exactamente 2 muones') 
                
                if(muons[0].GetCharge()*muons[1].GetCharge() > 0): continue #Carga opuesta
                update_cut_flows(signal, 'Carga opuesta') 
                
                if (muons[0].pt() < 30): continue 
                update_cut_flows(signal, 'p_T[0] > 30 GeV') 
                
                if (muons[1].pt() < 20): continue 
                update_cut_flows(signal, 'p_T[1] > 20 GeV') 

                if(abs(muons[0].eta()) > 2.4 or abs(muons[1].eta()) > 2.4): continue
                update_cut_flows(signal, '|Eta| < 2.4 ') 
                
                if(muons[0].DeltaR(muons[1]) < 0.3): continue
                update_cut_flows(signal, 'DeltaR > 0.3') 
                
                #Extraígamos las filas con las variables cinemáticas de cada muon, las variables que los correlacionan, para esto simplemente hay que usar la función get_kinematics_row definida en Delphes_Reader
                inf_muon0 = root_analysis.get_kinematics_row(muons[0])
                inf_muon1 = root_analysis.get_kinematics_row(muons[1])
                inf_correlation = root_analysis.get_kinematics_row(muons[1], muons[0])  
                
                #Reconstruyamos el z 
                Z = muons[0].GetTLV() + muons[1].GetTLV() #Extraígamos sus variables cinemáticas, aqui no podemos usar get_kinematics_row debido a que Z no es un elemento de la clase particulas, como tal no esta definido en la clase particulas, es un TLV que tiene las variables cinematicas de la particula reconstruida, por esto, debemos definir nosotros mismos el diccionario:            
                inf_Z = {"pT_{Z}(GeV)": Z.Pt(), "#eta_{Z}": Z.Eta(), "#phi_{Z}": Z.Phi(), "Energy_{Z}(GeV)": Z.Energy(), "Mass_{Z}(GeV)" : Z.M()} #Así queda en el mismo formato que las filas que extrajimos usando get_kinematics_row
                                
                #Añadamos esas filas a los DataFrames que se crearon en el primer for            
                row_muon0 = pd.DataFrame.from_dict(inf_muon0, orient ='index').T
                Data_muon0 = pd.concat([Data_muon0, row_muon0])                
                row_muon1 = pd.DataFrame.from_dict(inf_muon1, orient ='index').T
                Data_muon1 = pd.concat([Data_muon1, row_muon1])                
                row_correlation = pd.DataFrame.from_dict(inf_correlation, orient ='index').T
                Data_correlation = pd.concat([Data_correlation, row_correlation])
                row_Z = pd.DataFrame.from_dict(inf_Z, orient ='index').T
                Data_Z = pd.concat([Data_Z, row_Z])                  
            
            #En este punto ya se recorrieron todos los .root de la señal y se lleno el DataFrame, guardemoslo en un CSV
                
            Data_muon0.to_csv(f'{personal_folder}/CSV_Z_Analisis/Data_muon0/{signal}.csv')
            Data_muon1.to_csv(f'{personal_folder}/CSV_Z_Analisis/Data_muon1/{signal}.csv')
            Data_correlation.to_csv(f'{personal_folder}/CSV_Z_Analisis/Data_correlation/{signal}.csv')
            Data_Z.to_csv(f'{personal_folder}/CSV_Z_Analisis/Data_Z/{signal}.csv')
            #break #Asi solo se corre la primera path de cada señal
            
!touch CSVs_generados.txt #Testigo de que el código ya acabo de correr            

Aquí ya se generarán los CSV de los datos, guardemos también el DataFrame con la información de los cortes:

In [10]:
pd.DataFrame(cut_flows)

,Total,z,w_jets,ww,wz,zz,ttbar,stop
Todos,89400073,300000,20942823,12500000,9850000,10000000,24307250,11500000
Al menos 2 muones,936808,210853,104,74404,160145,299277,170215,21810
Exactamente 2 muones,916832,210828,97,74363,148789,291091,169923,21741
Carga opuesta,907174,210821,77,74288,142757,289570,168424,21237
p_T[0] > 30 GeV,853390,190460,26,66691,138989,282760,154900,19564
p_T[1] > 20 GeV,753660,183580,10,54154,123123,251079,125848,15866
|Eta| < 2.4,753660,183580,10,54154,123123,251079,125848,15866
DeltaR > 0.3,751645,183580,6,53657,123001,250981,124738,15682


In [11]:
pd.DataFrame(cut_flows).to_csv(f'{personal_folder}/CSV_Z_Analisis/cut_flows.csv')

Lo realizado a lo largo de este capítulo puede ocupar bastante tiempo, por eso es importante generar los archivos .csv, puesto que estos se leen mucho más rápido que los .root 